This script compiles stock market data from indexes names in a csv file placed on working diretory and additional indexes as function input. Data is fetched from Alpha Vantage so you need to get a key in order to run the code. The current example assembles a dataset containing all brazilian ADRs (from de csv file) and a few other ETFs (code input) with 15 minutes frequency (5, 10 and 30 are available). The script takes a while to run since it assumes the user is using a free Alpha Vantage key so it needs to wait a few seconds between data calls. 

In [ ]:
from pybovespa.bovespa import *
from pybovespa.stock import *
import matplotlib.pyplot as plt
import alpha_vantage as av
from pandas_datareader import data as pdr
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import requests
import time
import os
from datetime import datetime
os.chdir('/Users/marceloalleo/documents/Python/Trade')

In [112]:
key='your AV key here'

In [113]:
def get_ADRs ():
    return pd.read_csv('./ADRs.csv', sep= "\n")

In [238]:
def get_series_interval (name, series, interval = '15'):
    interval_values = []
    interval_indexes = []
    for index in series.index:
        string = index.split(':')
        if interval == '30':
            if string[1] == '00'or string[1] == '30':
                interval_values.append(series['close'][index])
                interval_indexes.append(index)
        elif interval == '15':
            if string[1] == '00'or string[1] == '15' or string[1] == '30' or string[1] == '45':
                interval_values.append(series['close'][index])
                interval_indexes.append(index)
        elif interval == '10':
            if string[1][1] == '0':
                interval_values.append(series['close'][index])
                interval_indexes.append(index)
        elif interval == '05':
            if string[1][1] == '0' or string[1][1] == '5':
                interval_values.append(series['close'][index])
                interval_indexes.append(index)
        else:
            print('Please type a valid interval (05, 10, 15 or 30)')
            return
    interval_series = pd.DataFrame([], index = interval_indexes)
    interval_series[name] = interval_values
    return interval_series

In [259]:
def get_table(add_indexes = [], interval_table = '15', CSV = pd.DataFrame([], columns = ['Codes'])):
    dataset = pd.DataFrame([])
    flag = 0
    ADRs = get_ADRs()
    for index in add_indexes:
        series = web.DataReader(index, "av-intraday", api_key = key)
        if flag == 0:
            dataset = get_series_interval(index, series, interval = interval_table)
            flag = 1
        else:
            dataset[index] = get_series_interval(index, series, interval = interval_table)[index]
        print(index)
        time.sleep(5)
    for index in CSV['Codes']:
        try:
            series = web.DataReader(index, "av-intraday", api_key = key)
            if flag == 0:
                dataset = get_series_interval(index, series, interval = interval_table)
                flag = 1
            else:
                dataset[index] = get_series_interval(index, series, interval = interval_table)[index]
            print(index)
        except:
            pass
        time.sleep(5)
    return dataset

In [260]:
dataset = get_table(['UUP', 'VXX', 'MCHI', 'IEUR'], CSV = get_ADRs())

UUP 773
VXX 1168
MCHI 1181
IEUR 554
ABEV 1223
BSBR 980
LND 19
BAK 368
BRFS 958
EBR 846
ERJ 1076
GGB 1120
GOL 1141
ITUB 1217
SUZ 760
VIV 1136
TSU 1112
UGP 934
VALE 1302
XP 1006


In [261]:
dataset

,UUP,VXX,MCHI,IEUR,ABEV,BSBR,LND,BAK,BRFS,EBR,ERJ,GGB,GOL,ITUB,SUZ,VIV,TSU,UGP,VALE,XP
2020-07-22 09:30:00,25.6400,NaN,72.510,45.5700,NaN,NaN,NaN,NaN,NaN,NaN,6.4300,NaN,NaN,5.3200,NaN,10.1800,NaN,NaN,11.7000,NaN
2020-07-22 09:45:00,25.6300,29.3400,72.280,NaN,2.935,5.8700,NaN,NaN,4.0666,7.530,6.4600,3.1600,7.9100,5.2700,NaN,10.1700,16.0100,3.8400,11.5950,45.4500
2020-07-22 10:00:00,25.6450,29.6600,72.380,45.6300,2.929,5.9500,NaN,9.570,4.1000,7.580,6.4550,3.2000,7.9600,5.3400,7.9000,10.1600,16.0000,3.9000,11.6800,46.2750
2020-07-22 10:15:00,25.6600,29.7700,72.330,NaN,2.950,5.9100,NaN,NaN,4.1200,7.540,6.4800,3.2251,7.9700,5.3100,7.9200,10.1000,15.9200,3.8800,11.6650,46.1000
2020-07-22 10:30:00,25.6550,30.0500,72.210,45.5600,2.950,5.8800,NaN,NaN,4.0900,NaN,6.4750,3.2050,7.9700,5.2850,7.8600,10.0499,15.8200,3.8990,11.6300,45.6900
2020-07-22 10:45:00,25.6450,29.5900,72.230,NaN,2.960,NaN,NaN,NaN,4.1000,7.570,6.5000,3.2350,7.9750,5.2950,7.8250,10.0600,15.8600,3.8950,11.6450,45.7800
2020-07-22 11:00:00,25.6300,29.7800,72.060,45.6000,2.990,5.9200,NaN,NaN,4.0900,7.650,6.5300,3.2150,7.9800,5.3150,7.7900,NaN,15.7900,NaN,11.6400,45.9900
2020-07-22 11:30:00,25.6250,29.6500,72.150,45.5900,2.970,5.8550,NaN,9.480,4.0700,NaN,6.4850,NaN,7.9250,5.2800,NaN,10.0100,15.8100,3.8400,11.6550,45.8600
2020-07-22 12:00:00,25.6350,29.5100,72.065,45.6000,2.975,5.8700,NaN,NaN,NaN,7.605,6.4550,3.2600,7.9200,5.3050,7.7400,10.0250,15.8050,3.8450,11.6050,45.7850
2020-07-22 12:45:00,25.6450,29.3000,71.940,NaN,2.940,5.8300,NaN,NaN,4.0400,7.600,6.4800,3.2650,7.9550,5.2850,7.7000,10.0050,15.7200,3.8300,11.6300,NaN
